In [1]:
import os
import pandas as pd
os.chdir('../../../')

from operations.api import KGFarm
kgfarm = KGFarm()

(KGFarm is running in Human in the loop mode)
Connected to Stardog!
Access the Stardog UI at: https://cloud.stardog.com/


In [2]:
entity_df = pd.read_csv(r'C:\Users\niki_\Google Drive\GRAD SCHOOL\Papers\KGFarm-LFE\Regression\forestfires.csv')
entity_df

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,4,3,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44
513,2,4,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29
514,7,4,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16
515,1,4,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00


In [3]:
# kgfarm.recommend_cleaning_operations(entity_df)

In [4]:
# entity_df = kgfarm.clean(entity_df,'fill')
# entity_df

In [5]:
# entity_df

# Without transformation

In [6]:
dependent_variable = "area"
independent_variables = [feature for feature in list(entity_df.columns) if feature != dependent_variable]

X = entity_df[independent_variables]
y = entity_df[dependent_variable]

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

### To convert date to float 

In [8]:
tranformation_info = kgfarm.recommend_feature_transformations(X_train)
tranformation_info

Processing unseen data


,Transformation,Package,Library,Feature,Transformation_type
0,MinMaxScaler,preprocessing,sklearn,"[X, ISI, rain]",Normalization
1,OrdinalEncoder,preprocessing,sklearn,"[month, day]",Ordinal encoding


In [9]:
X_train, _ = kgfarm.apply_transformation(tranformation_info.iloc[1], X_train)
X_train
X_test, _ = kgfarm.apply_transformation(tranformation_info.iloc[1], X_test)
X_test

Applying OrdinalEncoder transformation
2 feature(s) ['month', 'day'] transformed successfully!
Applying OrdinalEncoder transformation
2 feature(s) ['month', 'day'] transformed successfully!


,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain
213,7,3,6.0,1.0,87.6,52.2,103.8,5.0,11.0,46,5.8,0.0
435,2,5,4.0,2.0,90.8,84.7,376.6,5.6,23.8,51,1.8,0.0
343,8,6,9.0,1.0,91.5,130.1,807.1,7.5,15.9,51,4.5,0.0
230,4,4,9.0,6.0,92.9,133.3,699.6,9.2,26.4,21,4.5,0.0
171,8,6,1.0,6.0,85.6,90.4,609.6,6.6,17.4,50,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
26,7,4,9.0,0.0,92.4,117.9,668.0,12.2,19.0,34,5.8,0.0
208,6,5,9.0,0.0,92.4,117.9,668.0,12.2,19.6,33,6.3,0.0
4,8,6,6.0,3.0,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0
331,7,4,9.0,2.0,91.2,124.4,795.3,8.5,21.5,28,4.5,0.0


In [10]:
# from sklearn.linear_model import RidgeCV
# ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10.0),scoring='neg_mean_absolute_error')
# ridge_cv_model.fit(X_train,y_train)
# y_pred = ridge_cv_model.predict(X_test)

In [11]:
from sklearn.linear_model import Ridge
model = Ridge(alpha=1)

In [12]:
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

In [13]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
MAE = mean_absolute_error(y_test,y_pred)
MAE

21.880610204479183

In [14]:
MSE = mean_squared_error(y_test,y_pred)
RMSE = np.sqrt(MSE)
RMSE

90.5000998092002

# With transformation

In [15]:
X_transformed = entity_df[independent_variables]
y = entity_df[dependent_variable]

### Split train-test data

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.3, random_state=0)

### Transformation suggestions based on the training set

In [17]:
tranformation_info = kgfarm.recommend_feature_transformations(X_train)
tranformation_info

Processing unseen data


,Transformation,Package,Library,Feature,Transformation_type
0,MinMaxScaler,preprocessing,sklearn,"[X, ISI, rain]",Normalization
1,OrdinalEncoder,preprocessing,sklearn,"[month, day]",Ordinal encoding


### Transformation for Training set 

In [18]:
X_train, _ = kgfarm.apply_transformation(tranformation_info.iloc[0], X_train)
X_train

Applying MinMaxScaler transformation
3 feature(s) ['X', 'ISI', 'rain'] transformed successfully!


,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain
390,0.750,4,feb,mon,84.7,9.5,58.3,0.073084,7.5,71,6.3,0.0
5,0.875,6,aug,sun,92.3,85.3,488.0,0.262032,22.2,29,5.4,0.0
141,0.000,2,aug,wed,95.5,99.9,513.3,0.235294,23.3,31,4.5,0.0
268,0.250,4,aug,tue,92.1,152.6,658.2,0.254902,21.0,32,3.1,0.0
135,0.250,5,aug,sat,93.5,139.4,594.2,0.361854,17.6,52,5.8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
70,0.375,5,mar,fri,91.2,48.3,97.8,0.222816,15.8,27,7.6,0.0
277,0.250,4,dec,mon,85.4,25.4,349.7,0.046346,4.6,21,8.5,0.0
9,0.750,5,sep,sat,92.5,88.0,698.6,0.126560,22.8,40,4.0,0.0
359,0.000,4,sep,fri,92.5,122.0,789.7,0.181818,21.1,39,2.2,0.0


In [19]:
X_train, _ = kgfarm.apply_transformation(tranformation_info.iloc[1], X_train)
X_train

Applying OrdinalEncoder transformation
2 feature(s) ['month', 'day'] transformed successfully!


,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain
390,0.750,4,3.0,1.0,84.7,9.5,58.3,0.073084,7.5,71,6.3,0.0
5,0.875,6,1.0,3.0,92.3,85.3,488.0,0.262032,22.2,29,5.4,0.0
141,0.000,2,1.0,6.0,95.5,99.9,513.3,0.235294,23.3,31,4.5,0.0
268,0.250,4,1.0,5.0,92.1,152.6,658.2,0.254902,21.0,32,3.1,0.0
135,0.250,5,1.0,2.0,93.5,139.4,594.2,0.361854,17.6,52,5.8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
70,0.375,5,7.0,0.0,91.2,48.3,97.8,0.222816,15.8,27,7.6,0.0
277,0.250,4,2.0,1.0,85.4,25.4,349.7,0.046346,4.6,21,8.5,0.0
9,0.750,5,10.0,2.0,92.5,88.0,698.6,0.126560,22.8,40,4.0,0.0
359,0.000,4,10.0,0.0,92.5,122.0,789.7,0.181818,21.1,39,2.2,0.0


### Transformation on Testing set 

In [20]:
X_test, _ = kgfarm.apply_transformation(tranformation_info.iloc[0], X_test)
X_test

Applying MinMaxScaler transformation
3 feature(s) ['X', 'ISI', 'rain'] transformed successfully!


,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain
213,0.750,3,mar,mon,87.6,52.2,103.8,0.261364,11.0,46,5.8,0.0
435,0.125,5,jul,sat,90.8,84.7,376.6,0.295455,23.8,51,1.8,0.0
343,0.875,6,sep,mon,91.5,130.1,807.1,0.403409,15.9,51,4.5,0.0
230,0.375,4,sep,wed,92.9,133.3,699.6,0.500000,26.4,21,4.5,0.0
171,0.875,6,aug,wed,85.6,90.4,609.6,0.352273,17.4,50,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
26,0.750,4,sep,fri,92.4,117.9,668.0,0.670455,19.0,34,5.8,0.0
208,0.625,5,sep,fri,92.4,117.9,668.0,0.670455,19.6,33,6.3,0.0
4,0.875,6,mar,sun,89.3,51.3,102.2,0.522727,11.4,99,1.8,0.0
331,0.750,4,sep,sat,91.2,124.4,795.3,0.460227,21.5,28,4.5,0.0


In [21]:
X_test, _ = kgfarm.apply_transformation(tranformation_info.iloc[1], X_test)
X_test

Applying OrdinalEncoder transformation
2 feature(s) ['month', 'day'] transformed successfully!


,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain
213,0.750,3,6.0,1.0,87.6,52.2,103.8,0.261364,11.0,46,5.8,0.0
435,0.125,5,4.0,2.0,90.8,84.7,376.6,0.295455,23.8,51,1.8,0.0
343,0.875,6,9.0,1.0,91.5,130.1,807.1,0.403409,15.9,51,4.5,0.0
230,0.375,4,9.0,6.0,92.9,133.3,699.6,0.500000,26.4,21,4.5,0.0
171,0.875,6,1.0,6.0,85.6,90.4,609.6,0.352273,17.4,50,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
26,0.750,4,9.0,0.0,92.4,117.9,668.0,0.670455,19.0,34,5.8,0.0
208,0.625,5,9.0,0.0,92.4,117.9,668.0,0.670455,19.6,33,6.3,0.0
4,0.875,6,6.0,3.0,89.3,51.3,102.2,0.522727,11.4,99,1.8,0.0
331,0.750,4,9.0,2.0,91.2,124.4,795.3,0.460227,21.5,28,4.5,0.0


### Machine Learning Model

In [22]:
# from sklearn.linear_model import RidgeCV
# ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10.0),scoring='neg_mean_absolute_error')
# ridge_cv_model.fit(X_train,y_train)
# y_pred = ridge_cv_model.predict(X_test)

In [23]:
from sklearn.linear_model import Ridge
model = Ridge(alpha=1)

In [24]:
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

In [25]:
# ridge_cv_model.alpha_

In [26]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
MAE = mean_absolute_error(y_test,y_pred)
MAE

20.981243621967902

In [27]:
MSE = mean_squared_error(y_test,y_pred)
RMSE = np.sqrt(MSE)
RMSE

90.83228065556874